In [1]:

import cv2
import mediapipe as mp
import numpy as np
import json

class DanceTracker:
    def __init__(self, video_path):
        self.video_path = video_path
        self.mp_pose = mp.solutions.pose
        self.mp_drawing = mp.solutions.drawing_utils
        self.pose = self.mp_pose.Pose(
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )
        self.movement_data = []

    def calculate_velocity(self, prev_landmarks, curr_landmarks):
        """Calculate movement velocity between frames"""
        if prev_landmarks is None:
            return 0

        total_movement = 0
        for i in range(len(curr_landmarks)):
            prev = np.array([prev_landmarks[i].x, prev_landmarks[i].y])
            curr = np.array([curr_landmarks[i].x, curr_landmarks[i].y])
            total_movement += np.linalg.norm(curr - prev)

        return total_movement

    def get_joint_angles(self, landmarks):
        """Calculate key joint angles"""
        def angle_between(p1, p2, p3):
            v1 = np.array([p1.x - p2.x, p1.y - p2.y])
            v2 = np.array([p3.x - p2.x, p3.y - p2.y])

            cos_angle = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2) + 1e-6)
            angle = np.arccos(np.clip(cos_angle, -1.0, 1.0))
            return np.degrees(angle)

        angles = {}
        # Right elbow
        angles['right_elbow'] = angle_between(
            landmarks[self.mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
            landmarks[self.mp_pose.PoseLandmark.RIGHT_ELBOW.value],
            landmarks[self.mp_pose.PoseLandmark.RIGHT_WRIST.value]
        )
        # Left elbow
        angles['left_elbow'] = angle_between(
            landmarks[self.mp_pose.PoseLandmark.LEFT_SHOULDER.value],
            landmarks[self.mp_pose.PoseLandmark.LEFT_ELBOW.value],
            landmarks[self.mp_pose.PoseLandmark.LEFT_WRIST.value]
        )
        # Right knee
        angles['right_knee'] = angle_between(
            landmarks[self.mp_pose.PoseLandmark.RIGHT_HIP.value],
            landmarks[self.mp_pose.PoseLandmark.RIGHT_KNEE.value],
            landmarks[self.mp_pose.PoseLandmark.RIGHT_ANKLE.value]
        )
        # Left knee
        angles['left_knee'] = angle_between(
            landmarks[self.mp_pose.PoseLandmark.LEFT_HIP.value],
            landmarks[self.mp_pose.PoseLandmark.LEFT_KNEE.value],
            landmarks[self.mp_pose.PoseLandmark.LEFT_ANKLE.value]
        )

        return angles

    def process_video(self, output_path='output_dance.mp4', save_data=True):
        """Process video and track dance movements"""
        cap = cv2.VideoCapture(self.video_path)

        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = int(cap.get(cv2.CAP_PROP_FPS))

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

        frame_count = 0
        prev_landmarks = None

        print("Processing video...")

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame_count += 1
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = self.pose.process(rgb_frame)

            if results.pose_landmarks:
                # Draw pose landmarks
                self.mp_drawing.draw_landmarks(
                    frame,
                    results.pose_landmarks,
                    self.mp_pose.POSE_CONNECTIONS,
                    self.mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                    self.mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2)
                )

                landmarks = results.pose_landmarks.landmark

                # Calculate velocity
                velocity = self.calculate_velocity(prev_landmarks, landmarks)

                # Get joint angles
                angles = self.get_joint_angles(landmarks)

                # Store movement data
                frame_data = {
                    'frame': frame_count,
                    'velocity': float(velocity),
                    'angles': angles,
                    'timestamp': frame_count / fps
                }
                self.movement_data.append(frame_data)

                # Display info on frame
                cv2.putText(frame, f'Frame: {frame_count}', (10, 30),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                cv2.putText(frame, f'Movement: {velocity:.3f}', (10, 60),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

                prev_landmarks = landmarks

            out.write(frame)

            if frame_count % 30 == 0:
                print(f"Processed {frame_count} frames...")

        cap.release()
        out.release()

        print(f"\nProcessing complete!")
        print(f"Output saved to: {output_path}")

        # Save movement data
        if save_data:
            data_file = 'movement_data.json'
            with open(data_file, 'w') as f:
                json.dump(self.movement_data, f, indent=2)
            print(f"Movement data saved to: {data_file}")

        self.analyze_movements()

    def analyze_movements(self):
        """Analyze and summarize movement patterns"""
        if not self.movement_data:
            print("No movement data to analyze")
            return

        velocities = [d['velocity'] for d in self.movement_data]

        print("\n=== Movement Analysis ===")
        print(f"Total frames analyzed: {len(self.movement_data)}")
        print(f"Average movement velocity: {np.mean(velocities):.4f}")
        print(f"Peak movement velocity: {np.max(velocities):.4f}")
        print(f"Min movement velocity: {np.min(velocities):.4f}")

        # Find most active moments
        sorted_data = sorted(self.movement_data, key=lambda x: x['velocity'], reverse=True)
        print("\nTop 5 most active moments (by timestamp):")
        for i, data in enumerate(sorted_data[:5], 1):
            print(f"  {i}. {data['timestamp']:.2f}s - velocity: {data['velocity']:.4f}")

# Usage
if __name__ == "__main__":
    tracker = DanceTracker('dance.mp4')
    tracker.process_video(output_path='tracked_dance.mp4', save_data=True)

Processing video...
Processed 30 frames...
Processed 60 frames...
Processed 90 frames...
Processed 120 frames...
Processed 150 frames...
Processed 180 frames...
Processed 210 frames...
Processed 240 frames...
Processed 270 frames...
Processed 300 frames...
Processed 330 frames...
Processed 360 frames...
Processed 390 frames...
Processed 420 frames...
Processed 450 frames...
Processed 480 frames...
Processed 510 frames...
Processed 540 frames...
Processed 570 frames...
Processed 600 frames...
Processed 630 frames...
Processed 660 frames...
Processed 690 frames...
Processed 720 frames...
Processed 750 frames...
Processed 780 frames...
Processed 810 frames...
Processed 840 frames...
Processed 870 frames...
Processed 900 frames...
Processed 930 frames...
Processed 960 frames...
Processed 990 frames...
Processed 1020 frames...
Processed 1050 frames...
Processed 1080 frames...
Processed 1110 frames...
Processed 1140 frames...
Processed 1170 frames...
Processed 1200 frames...
Processed 1230 f

In [2]:
import cv2
import mediapipe as mp
import numpy as np
import json
from collections import defaultdict

class MultiPersonDanceTracker:
    def __init__(self, video_path):
        self.video_path = video_path
        self.mp_pose = mp.solutions.pose
        self.mp_drawing = mp.solutions.drawing_utils
        # Enable tracking for multiple people
        self.pose = self.mp_pose.Pose(
            static_image_mode=False,
            model_complexity=2,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )
        self.movement_data = defaultdict(list)
        self.person_colors = {}

    def generate_color(self, person_id):
        """Generate unique color for each person"""
        if person_id not in self.person_colors:
            np.random.seed(person_id * 42)
            self.person_colors[person_id] = tuple(np.random.randint(50, 255, 3).tolist())
        return self.person_colors[person_id]

    def calculate_velocity(self, prev_landmarks, curr_landmarks):
        """Calculate movement velocity between frames"""
        if prev_landmarks is None:
            return 0

        total_movement = 0
        for i in range(min(len(prev_landmarks), len(curr_landmarks))):
            prev = np.array([prev_landmarks[i].x, prev_landmarks[i].y])
            curr = np.array([curr_landmarks[i].x, curr_landmarks[i].y])
            total_movement += np.linalg.norm(curr - prev)

        return total_movement

    def get_bounding_box(self, landmarks, width, height):
        """Get bounding box for a person"""
        x_coords = [lm.x * width for lm in landmarks]
        y_coords = [lm.y * height for lm in landmarks]

        return {
            'x': int(min(x_coords)),
            'y': int(min(y_coords)),
            'w': int(max(x_coords) - min(x_coords)),
            'h': int(max(y_coords) - min(y_coords)),
            'center_x': int(np.mean(x_coords)),
            'center_y': int(np.mean(y_coords))
        }

    def get_joint_angles(self, landmarks):
        """Calculate key joint angles"""
        def angle_between(p1, p2, p3):
            v1 = np.array([p1.x - p2.x, p1.y - p2.y])
            v2 = np.array([p3.x - p2.x, p3.y - p2.y])

            cos_angle = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2) + 1e-6)
            angle = np.arccos(np.clip(cos_angle, -1.0, 1.0))
            return np.degrees(angle)

        angles = {}
        try:
            angles['right_elbow'] = angle_between(
                landmarks[self.mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                landmarks[self.mp_pose.PoseLandmark.RIGHT_ELBOW.value],
                landmarks[self.mp_pose.PoseLandmark.RIGHT_WRIST.value]
            )
            angles['left_elbow'] = angle_between(
                landmarks[self.mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                landmarks[self.mp_pose.PoseLandmark.LEFT_ELBOW.value],
                landmarks[self.mp_pose.PoseLandmark.LEFT_WRIST.value]
            )
            angles['right_knee'] = angle_between(
                landmarks[self.mp_pose.PoseLandmark.RIGHT_HIP.value],
                landmarks[self.mp_pose.PoseLandmark.RIGHT_KNEE.value],
                landmarks[self.mp_pose.PoseLandmark.RIGHT_ANKLE.value]
            )
            angles['left_knee'] = angle_between(
                landmarks[self.mp_pose.PoseLandmark.LEFT_HIP.value],
                landmarks[self.mp_pose.PoseLandmark.LEFT_KNEE.value],
                landmarks[self.mp_pose.PoseLandmark.LEFT_ANKLE.value]
            )
        except:
            pass

        return angles

    def match_person(self, bbox, prev_people, threshold=100):
        """Match detected person to previous frame using position"""
        if not prev_people:
            return len(prev_people)

        min_dist = float('inf')
        matched_id = None

        for person_id, prev_bbox in prev_people.items():
            dist = np.sqrt(
                (bbox['center_x'] - prev_bbox['center_x'])**2 +
                (bbox['center_y'] - prev_bbox['center_y'])**2
            )
            if dist < min_dist and dist < threshold:
                min_dist = dist
                matched_id = person_id

        if matched_id is None:
            matched_id = max(prev_people.keys()) + 1 if prev_people else 0

        return matched_id

    def process_video(self, output_path='output_multiperson_dance.mp4', save_data=True):
        """Process video and track multiple dancers"""
        cap = cv2.VideoCapture(self.video_path)

        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = int(cap.get(cv2.CAP_PROP_FPS))

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

        frame_count = 0
        prev_people = {}
        prev_landmarks = {}

        print("Processing video with multiple person tracking...")
        print("Note: For best results with many people, consider using YOLO + pose estimation")

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame_count += 1

            # Process frame in tiles for better multi-person detection
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Detect poses (MediaPipe detects one person per frame)
            # For true multi-person, we'd need to split frame or use YOLO
            results = self.pose.process(rgb_frame)

            current_people = {}

            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark
                bbox = self.get_bounding_box(landmarks, width, height)

                # Match to existing person
                person_id = self.match_person(bbox, prev_people)
                current_people[person_id] = bbox

                # Get color for this person
                color = self.generate_color(person_id)

                # Draw pose landmarks
                self.mp_drawing.draw_landmarks(
                    frame,
                    results.pose_landmarks,
                    self.mp_pose.POSE_CONNECTIONS,
                    self.mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=2),
                    self.mp_drawing.DrawingSpec(color=color, thickness=2)
                )

                # Calculate velocity
                velocity = self.calculate_velocity(
                    prev_landmarks.get(person_id),
                    landmarks
                )

                # Get joint angles
                angles = self.get_joint_angles(landmarks)

                # Store movement data
                frame_data = {
                    'frame': frame_count,
                    'person_id': person_id,
                    'velocity': float(velocity),
                    'angles': angles,
                    'bbox': bbox,
                    'timestamp': frame_count / fps
                }
                self.movement_data[person_id].append(frame_data)

                # Draw bounding box and label
                cv2.rectangle(frame,
                            (bbox['x'], bbox['y']),
                            (bbox['x'] + bbox['w'], bbox['y'] + bbox['h']),
                            color, 2)
                cv2.putText(frame, f'Person {person_id}',
                           (bbox['x'], bbox['y'] - 10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                cv2.putText(frame, f'Vel: {velocity:.2f}',
                           (bbox['x'], bbox['y'] + bbox['h'] + 20),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

                prev_landmarks[person_id] = landmarks

            # Display frame info
            cv2.putText(frame, f'Frame: {frame_count} | People: {len(current_people)}',
                       (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

            prev_people = current_people
            out.write(frame)

            if frame_count % 30 == 0:
                print(f"Processed {frame_count} frames, tracking {len(self.movement_data)} people...")

        cap.release()
        out.release()

        print(f"\nProcessing complete!")
        print(f"Output saved to: {output_path}")
        print(f"Total people tracked: {len(self.movement_data)}")

        # Save movement data
        if save_data:
            data_file = 'multiperson_movement_data.json'
            with open(data_file, 'w') as f:
                json.dump(dict(self.movement_data), f, indent=2)
            print(f"Movement data saved to: {data_file}")

        self.analyze_movements()

    def analyze_movements(self):
        """Analyze and summarize movement patterns for all people"""
        if not self.movement_data:
            print("No movement data to analyze")
            return

        print("\n=== Multi-Person Movement Analysis ===")
        print(f"Total people tracked: {len(self.movement_data)}")

        for person_id, data in self.movement_data.items():
            velocities = [d['velocity'] for d in data]

            print(f"\n--- Person {person_id} ---")
            print(f"Frames tracked: {len(data)}")
            print(f"Average movement: {np.mean(velocities):.4f}")
            print(f"Peak movement: {np.max(velocities):.4f}")
            print(f"Total activity time: {len(data) / 30:.2f}s")

            # Find most active moment
            most_active = max(data, key=lambda x: x['velocity'])
            print(f"Most active at: {most_active['timestamp']:.2f}s")

        # Compare all dancers
        print("\n=== Dancer Comparison ===")
        avg_movements = {pid: np.mean([d['velocity'] for d in data])
                        for pid, data in self.movement_data.items()}

        sorted_dancers = sorted(avg_movements.items(), key=lambda x: x[1], reverse=True)
        print("Ranking by average movement intensity:")
        for rank, (pid, avg_vel) in enumerate(sorted_dancers, 1):
            print(f"  {rank}. Person {pid}: {avg_vel:.4f}")

# Usage
if __name__ == "__main__":
    tracker = MultiPersonDanceTracker('dance.mp4')
    tracker.process_video(output_path='tracked_multiperson_dance.mp4', save_data=True)

Processing video with multiple person tracking...
Note: For best results with many people, consider using YOLO + pose estimation
Processed 30 frames, tracking 4 people...
Processed 60 frames, tracking 4 people...
Processed 90 frames, tracking 4 people...
Processed 120 frames, tracking 4 people...
Processed 150 frames, tracking 4 people...
Processed 180 frames, tracking 4 people...
Processed 210 frames, tracking 4 people...
Processed 240 frames, tracking 4 people...
Processed 270 frames, tracking 4 people...
Processed 300 frames, tracking 4 people...
Processed 330 frames, tracking 4 people...
Processed 360 frames, tracking 4 people...
Processed 390 frames, tracking 4 people...
Processed 420 frames, tracking 4 people...
Processed 450 frames, tracking 4 people...
Processed 480 frames, tracking 4 people...
Processed 510 frames, tracking 4 people...
Processed 540 frames, tracking 4 people...
Processed 570 frames, tracking 4 people...
Processed 600 frames, tracking 4 people...
Processed 630 

In [3]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 107.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mediapipe 0.10.21 requires numpy<2, but you have numpy 2.2.6 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.2.6 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.6 which is incompatible.


In [2]:
import cv2
import mediapipe as mp
import numpy as np
import json
from collections import defaultdict
from ultralytics import YOLO
import os

class YOLODanceTracker:
    def __init__(self, video_path):
        self.video_path = video_path

        # Validate video file first
        if not self.validate_video():
            raise ValueError("Invalid or corrupted video file!")

        # Initialize YOLO for person detection
        print("Loading YOLO model...")
        self.yolo_model = YOLO('yolov8n.pt')

        # Initialize MediaPipe for detailed pose estimation
        print("Loading MediaPipe pose model...")
        self.mp_pose = mp.solutions.pose
        self.mp_drawing = mp.solutions.drawing_utils
        self.pose = self.mp_pose.Pose(
            static_image_mode=False,
            model_complexity=1,
            min_detection_confidence=0.6,
            min_tracking_confidence=0.6
        )

        self.movement_data = defaultdict(list)
        self.person_colors = {}
        self.prev_landmarks = {}

    def validate_video(self):
        """Validate video file before processing"""
        print("\n" + "=" * 50)
        print("VIDEO VALIDATION")
        print("=" * 50)

        # Check if file exists
        if not os.path.exists(self.video_path):
            print(f"❌ ERROR: File not found: {self.video_path}")
            return False

        # Check file size
        file_size = os.path.getsize(self.video_path)
        print(f"File size: {file_size:,} bytes ({file_size / 1024:.2f} KB)")

        if file_size < 10000:  # Less than 10KB is suspicious
            print("❌ ERROR: File is too small (likely corrupted or empty)")
            print("\nPossible issues:")
            print("  - Incomplete download")
            print("  - Corrupted file")
            print("  - Wrong file path")
            print("  - File is just metadata without video data")
            return False

        # Try to open video
        cap = cv2.VideoCapture(self.video_path)

        if not cap.isOpened():
            print("❌ ERROR: Cannot open video file")
            print("\nPossible issues:")
            print("  - Unsupported codec")
            print("  - Corrupted file structure")
            print("  - Missing codecs on system")
            cap.release()
            return False

        # Get video properties
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        duration = frame_count / fps if fps > 0 else 0

        print(f"\n✓ Video Properties:")
        print(f"  Resolution: {width}x{height}")
        print(f"  FPS: {fps}")
        print(f"  Total frames: {frame_count}")
        print(f"  Duration: {duration:.2f} seconds")

        # Try to read first frame
        ret, frame = cap.read()
        if not ret or frame is None:
            print("\n❌ ERROR: Cannot read video frames")
            print("The file exists but contains no readable video data")
            cap.release()
            return False

        print(f"  First frame shape: {frame.shape}")
        print("\n✓ Video file is valid!")
        cap.release()
        return True

    def generate_color(self, person_id):
        """Generate unique color for each person"""
        if person_id not in self.person_colors:
            np.random.seed(person_id * 42)
            self.person_colors[person_id] = tuple(np.random.randint(50, 255, 3).tolist())
        return self.person_colors[person_id]

    def calculate_velocity(self, prev_landmarks, curr_landmarks):
        """Calculate movement velocity between frames"""
        if prev_landmarks is None:
            return 0

        total_movement = 0
        for i in range(min(len(prev_landmarks), len(curr_landmarks))):
            prev = np.array([prev_landmarks[i].x, prev_landmarks[i].y])
            curr = np.array([curr_landmarks[i].x, curr_landmarks[i].y])
            total_movement += np.linalg.norm(curr - prev)

        return total_movement

    def get_joint_angles(self, landmarks):
        """Calculate key joint angles"""
        def angle_between(p1, p2, p3):
            v1 = np.array([p1.x - p2.x, p1.y - p2.y])
            v2 = np.array([p3.x - p2.x, p3.y - p2.y])

            cos_angle = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2) + 1e-6)
            angle = np.arccos(np.clip(cos_angle, -1.0, 1.0))
            return np.degrees(angle)

        angles = {}
        try:
            angles['right_elbow'] = angle_between(
                landmarks[self.mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                landmarks[self.mp_pose.PoseLandmark.RIGHT_ELBOW.value],
                landmarks[self.mp_pose.PoseLandmark.RIGHT_WRIST.value]
            )
            angles['left_elbow'] = angle_between(
                landmarks[self.mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                landmarks[self.mp_pose.PoseLandmark.LEFT_ELBOW.value],
                landmarks[self.mp_pose.PoseLandmark.LEFT_WRIST.value]
            )
            angles['right_knee'] = angle_between(
                landmarks[self.mp_pose.PoseLandmark.RIGHT_HIP.value],
                landmarks[self.mp_pose.PoseLandmark.RIGHT_KNEE.value],
                landmarks[self.mp_pose.PoseLandmark.RIGHT_ANKLE.value]
            )
            angles['left_knee'] = angle_between(
                landmarks[self.mp_pose.PoseLandmark.LEFT_HIP.value],
                landmarks[self.mp_pose.PoseLandmark.LEFT_KNEE.value],
                landmarks[self.mp_pose.PoseLandmark.LEFT_ANKLE.value]
            )
        except:
            pass

        return angles

    def calculate_energy_level(self, velocity, angles):
        """Calculate overall energy/intensity level"""
        energy = velocity * 10

        if angles:
            for angle in angles.values():
                if 30 < angle < 150:
                    energy += 0.5

        return energy

    def process_video(self, output_path='output_yolo_dance.mp4', save_data=True,
                     min_detection_confidence=0.7, max_people=50):
        """Process video with YOLO detection + MediaPipe pose"""
        cap = cv2.VideoCapture(self.video_path)

        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height + 60))

        frame_count = 0
        frame_skip = 0  # Process every frame

        print("\n" + "=" * 50)
        print("PROCESSING VIDEO")
        print("=" * 50)
        print(f"Video: {total_frames} frames at {fps} FPS")
        print(f"Resolution: {width}x{height}")
        print(f"Min confidence: {min_detection_confidence}")
        print(f"Max people to track: {max_people}")
        print("=" * 50 + "\n")

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame_count += 1

            # Skip frames for faster processing (optional)
            if frame_count % (frame_skip + 1) != 0:
                continue

            # YOLO detection with stricter settings
            results = self.yolo_model.track(
                frame,
                persist=True,
                classes=[0],  # Only detect persons
                conf=min_detection_confidence,
                iou=0.5,
                verbose=False
            )

            current_detections = []

            if results[0].boxes is not None and len(results[0].boxes) > 0:
                boxes = results[0].boxes

                # Limit number of tracked people
                if len(boxes) > max_people:
                    print(f"⚠️  Warning: {len(boxes)} people detected in frame {frame_count}, "
                          f"limiting to {max_people}")
                    boxes = boxes[:max_people]

                for i, box in enumerate(boxes):
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    confidence = float(box.conf[0])

                    # Get tracking ID
                    track_id = int(box.id[0]) if box.id is not None else None

                    if track_id is None or confidence < min_detection_confidence:
                        continue

                    # Validate bounding box size (filter out tiny detections)
                    box_width = x2 - x1
                    box_height = y2 - y1

                    if box_width < 30 or box_height < 50:  # Too small
                        continue

                    if box_width > width * 0.8 or box_height > height * 0.8:  # Too large
                        continue

                    # Crop person from frame
                    person_crop = frame[max(0, y1):min(height, y2),
                                       max(0, x1):min(width, x2)]

                    if person_crop.size == 0:
                        continue

                    # Run pose estimation
                    rgb_crop = cv2.cvtColor(person_crop, cv2.COLOR_BGR2RGB)
                    pose_results = self.pose.process(rgb_crop)

                    if pose_results.pose_landmarks:
                        landmarks = pose_results.pose_landmarks.landmark

                        velocity = self.calculate_velocity(
                            self.prev_landmarks.get(track_id),
                            landmarks
                        )

                        angles = self.get_joint_angles(landmarks)
                        energy = self.calculate_energy_level(velocity, angles)

                        # Store data
                        frame_data = {
                            'frame': frame_count,
                            'person_id': track_id,
                            'velocity': float(velocity),
                            'energy': float(energy),
                            'angles': angles,
                            'bbox': {'x': x1, 'y': y1, 'w': box_width, 'h': box_height},
                            'confidence': float(confidence),
                            'timestamp': frame_count / fps
                        }
                        self.movement_data[track_id].append(frame_data)

                        color = self.generate_color(track_id)

                        # Draw skeleton
                        for landmark in landmarks:
                            lx = int(landmark.x * box_width + x1)
                            ly = int(landmark.y * box_height + y1)
                            cv2.circle(frame, (lx, ly), 3, color, -1)

                        # Draw connections
                        for connection in self.mp_pose.POSE_CONNECTIONS:
                            start = landmarks[connection[0]]
                            end = landmarks[connection[1]]

                            start_x = int(start.x * box_width + x1)
                            start_y = int(start.y * box_height + y1)
                            end_x = int(end.x * box_width + x1)
                            end_y = int(end.y * box_height + y1)

                            cv2.line(frame, (start_x, start_y), (end_x, end_y),
                                   color, 2)

                        # Draw bounding box and labels
                        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                        cv2.putText(frame, f'ID:{track_id}', (x1, y1 - 10),
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                        cv2.putText(frame, f'E:{energy:.1f}', (x1, y2 + 20),
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

                        self.prev_landmarks[track_id] = landmarks
                        current_detections.append(track_id)

            # Add info bar
            info_bg = np.zeros((60, width, 3), dtype=np.uint8)
            cv2.putText(info_bg, f'Frame: {frame_count}/{total_frames}',
                       (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            cv2.putText(info_bg, f'Current: {len(current_detections)} | Total: {len(self.movement_data)}',
                       (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

            frame_with_info = np.vstack([info_bg, frame])
            out.write(frame_with_info)

            if frame_count % 30 == 0:
                progress = (frame_count / total_frames) * 100
                print(f"Progress: {progress:.1f}% | Frame {frame_count}/{total_frames} | "
                      f"Current: {len(current_detections)} dancers | "
                      f"Total tracked: {len(self.movement_data)}")

        cap.release()
        out.release()

        print("\n" + "=" * 50)
        print(f"✓ Processing complete!")
        print(f"✓ Output: {output_path}")
        print(f"✓ Unique dancers: {len(self.movement_data)}")
        print("=" * 50)

        if save_data:
            data_file = 'yolo_dance_data.json'
            with open(data_file, 'w') as f:
                json.dump(dict(self.movement_data), f, indent=2)
            print(f"✓ Data saved: {data_file}")

        if len(self.movement_data) > 0:
            self.analyze_movements()
            self.generate_summary()
        else:
            print("\n⚠️  WARNING: No dancers detected in video!")
            print("Possible issues:")
            print("  - Video quality too low")
            print("  - No people visible in frames")
            print("  - Detection confidence threshold too high")

    def analyze_movements(self):
        """Analyze movement patterns"""
        print("\n" + "=" * 50)
        print("INDIVIDUAL DANCER ANALYSIS")
        print("=" * 50)

        for person_id, data in sorted(self.movement_data.items()):
            velocities = [d['velocity'] for d in data]
            energies = [d['energy'] for d in data]

            print(f"\n--- Person {person_id} ---")
            print(f"Frames: {len(data)} | Time: {len(data) / 30:.2f}s")
            print(f"Avg velocity: {np.mean(velocities):.4f}")
            print(f"Avg energy: {np.mean(energies):.2f}")
            print(f"Peak energy: {np.max(energies):.2f}")

    def generate_summary(self):
        """Generate comparative summary"""
        print("\n" + "=" * 50)
        print("COMPARATIVE ANALYSIS")
        print("=" * 50)

        dancer_stats = {}
        for pid, data in self.movement_data.items():
            dancer_stats[pid] = {
                'avg_energy': np.mean([d['energy'] for d in data]),
                'screen_time': len(data) / 30,
                'total_frames': len(data)
            }

        print("\nTop 10 by Energy Level:")
        sorted_by_energy = sorted(dancer_stats.items(),
                                 key=lambda x: x[1]['avg_energy'],
                                 reverse=True)[:10]
        for rank, (pid, stats) in enumerate(sorted_by_energy, 1):
            print(f"  {rank}. Person {pid}: {stats['avg_energy']:.2f} "
                  f"(time: {stats['screen_time']:.1f}s)")

        print("\nTop 10 by Screen Time:")
        sorted_by_time = sorted(dancer_stats.items(),
                               key=lambda x: x[1]['screen_time'],
                               reverse=True)[:10]
        for rank, (pid, stats) in enumerate(sorted_by_time, 1):
            print(f"  {rank}. Person {pid}: {stats['screen_time']:.1f}s "
                  f"({stats['total_frames']} frames)")

# Usage
if __name__ == "__main__":
    try:
        tracker = YOLODanceTracker('dance.mp4')

        # Process with stricter settings to avoid false detections
        tracker.process_video(
            output_path='yolo_tracked_dance.mp4',
            save_data=True,
            min_detection_confidence=0.7,  # Higher confidence
            max_people=50  # Reasonable limit
        )

    except Exception as e:
        print(f"\n❌ ERROR: {str(e)}")
        print("\nTroubleshooting steps:")
        print("1. Check if dance.mp4 exists and is not corrupted")
        print("2. Try re-downloading or re-encoding the video")
        print("3. Test with a different video file")
        print("4. Verify video with: ffmpeg -i dance.mp4")


VIDEO VALIDATION
File size: 57,286,555 bytes (55943.90 KB)

✓ Video Properties:
  Resolution: 1914x800
  FPS: 30
  Total frames: 2078
  Duration: 69.27 seconds
  First frame shape: (800, 1914, 3)

✓ Video file is valid!
Loading YOLO model...
Loading MediaPipe pose model...

PROCESSING VIDEO
Video: 2078 frames at 30 FPS
Resolution: 1914x800
Min confidence: 0.7
Max people to track: 50

Progress: 1.4% | Frame 30/2078 | Current: 0 dancers | Total tracked: 2
Progress: 2.9% | Frame 60/2078 | Current: 0 dancers | Total tracked: 2
Progress: 4.3% | Frame 90/2078 | Current: 1 dancers | Total tracked: 2
Progress: 5.8% | Frame 120/2078 | Current: 1 dancers | Total tracked: 3
Progress: 7.2% | Frame 150/2078 | Current: 0 dancers | Total tracked: 3
Progress: 8.7% | Frame 180/2078 | Current: 0 dancers | Total tracked: 3
Progress: 10.1% | Frame 210/2078 | Current: 1 dancers | Total tracked: 6
Progress: 11.5% | Frame 240/2078 | Current: 2 dancers | Total tracked: 9
Progress: 13.0% | Frame 270/2078 | Cu